<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #19: Process Spyfish Aotearoa videos</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Nov 14th, 2021</h5>

# Set up and requirements

Import Python packages

In [1]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.tutorials_utils as t_utils
import utils.t19_utils as t19
#import utils.server_utils as serv_utils

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [2]:
project = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

### Initiate sql and get server or local storage details

In [3]:
# Initiate db
db_info_dict = t_utils.initiate_db(project.value)

Enter the key id for the aws server········
Enter the secret access key for the aws server········


../db_starter/db_csv_info/sites_buv_doc.csv: 100%|██████████| 4.24k/4.24k [00:00<00:00, 26.8kB/s]
../db_starter/db_csv_info/movies_buv_doc.csv: 100%|██████████| 7.25k/7.25k [00:00<00:00, 49.3kB/s]
../db_starter/db_csv_info/species_buv_doc.csv: 100%|██████████| 205/205 [00:00<00:00, 1.63kB/s]


Updated sites
Updated movies
Updated species


# Select the folder where the sd card is

Make sure you select the FOLDER not the files

In [4]:
go_pro_folder = t19.select_go_pro_folder()

FileChooser(path='C:\', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Change', …

In [5]:
go_pro_movies_i = t19.select_go_pro_movies(go_pro_folder.selected)
print("The movies selected are:")
print(*go_pro_movies_i, sep='\n')

The movies selected are:
C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\GH010925.MP4
C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\GH020925.MP4
C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\GH030925.MP4


### Select the date and site

In [6]:
site_i = t19.select_site(db_info_dict)
date_i = t19.select_date()

interactive(children=(Dropdown(description='Existing or new site:', layout=Layout(width='50%'), options=('Exis…

DatePicker(value=None, description='Date of recording', layout=Layout(width='50%'), style=DescriptionStyle(des…

### Concatenate the go pro files

In [7]:
video_info_dict = t19.concatenate_go_pro_videos(site_i.result.value,
                                                date_i.value.isoformat().replace("-","_"),
                                                go_pro_folder.selected,
                                                go_pro_movies_i)

Concatenating  C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\TUH_040_2021_11_11.MP4
C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\TUH_040_2021_11_11.MP4 concatenated successfully
Open C:\Users\Victor\Desktop\tuhua_buv_2021\WP40\TUH_040_2021_11_11.MP4 to check the required information for the next sections


### Select Author, bad deployment, survey start/end, s3 bucket

In [8]:
# Select author
video_author = t19.select_author(db_info_dict)

# Specify id deployment is bad
deployment_i = t19.select_bad_deployment()

# Set survey start
surv_start = t19.select_start_survey(video_info_dict["length"])

#TODO Set survey end
surv_end = t19.select_end_survey(video_info_dict["length"])

# Select the S3 "folder" to upload the video
s3_folder = t19.select_s3_folder(db_info_dict)

#Add any comment related to the movie
comment_movie = t19.write_comment()

interactive(children=(Dropdown(description='Existing or new Author:', layout=Layout(width='50%'), options=('Ex…

interactive(children=(Dropdown(description='Is it a bad deployment?', index=1, layout=Layout(width='50%'), opt…

interactive(children=(IntSlider(value=0, description='Survey starts (seconds):', layout=Layout(width='50%'), m…

interactive(children=(IntSlider(value=1717, description='Survey ends (seconds):', layout=Layout(width='50%'), …

Combobox(value='', description='S3 folder:', ensure_option=True, options=('ROV', 'Tonga Island BUV', 'bayofisl…

Text(value='', description='Comment:', layout=Layout(width='50%'), placeholder='Type comment', style=Descripti…

### Review the survey/movie details

In [9]:
new_row, movies_df = t19.review_movie_details(
    project_name = project.value,
    db_info_dict = db_info_dict,
    video_info_dict_i = video_info_dict,
    IsBadDeployment_i = deployment_i.result,
    survey_start_i = surv_start.result,
    survey_end_i = surv_end.result,
    author_i = video_author.result.value,
    comment_i = comment_movie.value,
    s3_prefix_i = s3_folder.value,
    go_pro_list_i = go_pro_movies_i,
    bucket_i = 'marine-buv'
)
print("The new details are:")
new_row.to_dict('records')

../db_starter/db_csv_info/sites_buv_doc.csv: 100%|██████████| 4.24k/4.24k [00:00<00:00, 28.3kB/s]
../db_starter/db_csv_info/movies_buv_doc.csv: 100%|██████████| 7.25k/7.25k [00:00<00:00, 50.4kB/s]
../db_starter/db_csv_info/species_buv_doc.csv: 100%|██████████| 205/205 [00:00<00:00, 1.67kB/s]

The new details are:


[{'movie_id': 58,
  'filename': 'TUH_040_2021_11_11.MP4',
  'siteName': 'TUH_040',
  'created_on': '2021_11_11',
  'Author': 'MoniqueLadds',
  'fps': 59.93971089507259,
  'duration': 1717.2588666666666,
  'survey_start': 160,
  'survey_end': 1650,
  'go_pro_files': ['C:\\Users\\Victor\\Desktop\\tuhua_buv_2021\\WP40\\GH010925.MP4',
   'C:\\Users\\Victor\\Desktop\\tuhua_buv_2021\\WP40\\GH020925.MP4',
   'C:\\Users\\Victor\\Desktop\\tuhua_buv_2021\\WP40\\GH030925.MP4'],
  'bucket': 'marine-buv',
  'prefix': 'poorknights-buv-2015/TUH_040_2021_11_11',
  'IsBadDeployment': False,
  'comment': 'this is just a trial'}]

# !!!Only pass this point if movie details are correct!!!

### Update movies csv and upload video to s3

In [ ]:
upload_concat_movie(new_row, db_info_dict, video_info_dict, movies_df)

In [ ]:
#END